In [1]:
import pandas as pd
import numpy as np

In [2]:
import pickle

In [3]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [4]:
from sklearn.decomposition import PCA

In [5]:
from sklearn.model_selection import GridSearchCV

In [3]:
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [4]:
def simple_weather(row):
    if row in ['Sunny', 'Clear']:
        return 'Sunny'
    
    elif row in ['Partly cloudy','Overcast','Cloudy']:
        return 'Cloudy'
    
    elif row in ['Torrential rain shower', 'Heavy rain', 'Heavy rain at times']:
        return 'Heavy_rain'

    elif row in ['Moderate rain', 'Moderate rain at times', 'Patchy rain possible',
                 'Light rain shower','Light drizzle','Light rain',
                 'Patchy light rain with thunder', 'Patchy light drizzle', 'Patchy light rain',
                 'Moderate or heavy rain with thunder']:
        return 'Light&Mod_rain'

    elif row in ['Moderate or heavy freezing rain']:
        return 'Freezing_rain'
    
    elif row in ['Mist','Fog', 'Freezing fog']:
        return 'Fog'
    
    elif row in ['Heavy snow', 'Moderate or heavy snow showers', 'Patchy heavy snow', 
                                  'Blizzard', 'Blowing snow', 'Ice pellets']:
        return 'Heavy_snow'
    
    elif row in ['Moderate snow', 'Moderate or heavy snow showers','Patchy moderate snow',
                                  'Moderate or heavy sleet', 'Moderate or heavy snow with thunder', 'Light snow', 'Light sleet', 'Patchy light snow', 'Light sleet showers',
                                  'Patchy sleet possible', 'Light snow showers', 
                                  'Patchy light snow with thunder', 'Patchy snow possible']:
        return 'Light&Mod_snow'
    

    elif row in ['Thundery outbreaks possible']:
        return 'Thunder'
    
'''
    elif row in ['Patchy rain possible','Light rain shower','Light drizzle','Light rain',
                                  'Patchy light rain with thunder', 'Patchy light drizzle', 'Patchy light rain']:
        return 'Light_rain
    elif row in ['Light snow', 'Light sleet', 'Patchy light snow', 'Light sleet showers',
                                  'Patchy sleet possible', 'Light snow showers', 
                                  'Patchy light snow with thunder', 'Patchy snow possible']:
        return 'Light_snow
'''
    

"\n    elif row in ['Patchy rain possible','Light rain shower','Light drizzle','Light rain',\n                                  'Patchy light rain with thunder', 'Patchy light drizzle', 'Patchy light rain']:\n        return 'Light_rain\n    elif row in ['Light snow', 'Light sleet', 'Patchy light snow', 'Light sleet showers',\n                                  'Patchy sleet possible', 'Light snow showers', \n                                  'Patchy light snow with thunder', 'Patchy snow possible']:\n        return 'Light_snow\n"

In [5]:
flights = pd.read_csv('flights_2019_passengers.csv')

In [36]:
flights.columns.values

array(['op_unique_carrier', 'origin', 'dest', 'crs_dep_time', 'dep_time',
       'taxi_out', 'wheels_off', 'wheels_on', 'taxi_in', 'crs_arr_time',
       'arr_time', 'arr_delay', 'crs_elapsed_time', 'actual_elapsed_time',
       'air_time', 'distance', 'dep_condition', 'arr_condition', 'month',
       'average_seats', 'average_passengers',
       'avg_passenger_capacity_usage'], dtype=object)

In [ ]:
'''location = ['origin', 'dest']
carrier =  ['mkt_unique_carrier', 'branded_code_share', 'mkt_carrier', 'mkt_carrier_fl_num', 'op_unique_carrier','tail_num', 'op_carrier_fl_num']
time24hr = ['crs_dep_time', 'dep_time', 'wheels_off', 'wheels_on', 'crs_arr_time', 'arr_time']
flighttime = ['crs_elapsed_time', 'actual_elapsed_time', 'air_time', 'taxi_out', 'taxi_in']
drop = ['no_name','dup','dest_city_name','dest_airport_id', 'origin_airport_id', 'origin_city_name', 'flights']
stopover = ['first_dep_time', 'total_add_gtime', 'longest_add_gtime']
cancel = ['cancelled', 'cancellation_code','diverted']
crs = ['crs_dep_time', 'crs_arr_time', 'crs_elapsed_time']
delays= [arr_delay, 'dep_delay', 'carrier_delay, 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay'] '''

In [37]:
flights_num.isna().sum()

crs_dep_time                                   0
dep_time                                     136
taxi_out                                     140
wheels_off                                   140
wheels_on                                    145
                                            ... 
arr_condition_Patchy rain possible             0
arr_condition_Patchy sleet possible            0
arr_condition_Sunny                            0
arr_condition_Thundery outbreaks possible      0
arr_condition_Torrential rain shower           0
Length: 750, dtype: int64

In [6]:
flights = flights.drop(columns=['no_name','dup','dest_city_name','dest_airport_id', 'origin_airport_id', 'origin_city_name', 'flights'])

In [7]:
flights = flights.drop(columns=['mkt_unique_carrier', 'branded_code_share', 'mkt_carrier', 'mkt_carrier_fl_num', 'tail_num', 'op_carrier_fl_num'])

In [8]:
flights = flights.drop(columns=['arr_date','fl_date'])

In [9]:
flights = flights.drop(columns=['first_dep_time', 'total_add_gtime', 'longest_add_gtime', 'cancelled', 'cancellation_code','diverted'])

In [10]:
flights = flights.drop(columns = ['year'])

In [11]:
flights = flights.drop(columns = ['wheels_off', 'wheels_on', 'average_seats'])

In [12]:
flights = flights.drop(columns = ['dep_delay', 'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay'])

In [13]:
flights['weather_origin_simple'] = flights.apply(lambda row: simple_weather(row['dep_condition']), axis=1)
flights['weather_dest_simple'] = flights.apply(lambda row: simple_weather(row['arr_condition']), axis=1)

In [14]:
flights = flights.drop(columns = ['dep_condition', 'arr_condition'])

In [14]:
#flights_samp = flights.sample(n=12000)

In [11]:
flights.head()

,op_unique_carrier,origin,dest,crs_dep_time,dep_time,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,...,crs_elapsed_time,actual_elapsed_time,air_time,distance,dep_condition,arr_condition,month,average_seats,average_passengers,avg_passenger_capacity_usage
0,OO,ORD,COS,955,1056.0,25.0,1121.0,1253.0,6.0,1152,...,177,183.0,152.0,911,Overcast,Partly cloudy,1,1002.000000,777.666667,0.776114
1,OH,MYR,CLT,2157,2158.0,10.0,2208.0,2248.0,20.0,2311,...,74,70.0,40.0,157,Cloudy,Sunny,1,6327.333333,4898.333333,0.774154
2,OH,BDL,DCA,1151,1212.0,7.0,1219.0,1328.0,3.0,1325,...,94,79.0,69.0,313,Sunny,Sunny,1,3516.500000,2697.250000,0.767027
3,OH,DCA,BDL,1009,1013.0,32.0,1045.0,1137.0,3.0,1126,...,77,87.0,52.0,313,Sunny,Sunny,1,3460.000000,2494.500000,0.720954
4,OH,DCA,GSP,2000,1955.0,13.0,2008.0,2120.0,2.0,2134,...,94,87.0,72.0,396,Sunny,Partly cloudy,1,1511.000000,847.333333,0.560777


In [15]:
flights= flights.dropna()


In [16]:
flights= flights.reset_index(drop=True)


In [17]:
delays = flights['arr_delay']

In [18]:
flights = flights.drop(columns = 'arr_delay')

In [22]:
delays.mean()

14.48021305550799

In [21]:
delays = delays.clip(lower=0)

In [23]:
y = delays

In [23]:
flights.dtypes

op_unique_carrier                object
origin                           object
dest                             object
crs_dep_time                      int64
dep_time                        float64
taxi_out                        float64
wheels_off                      float64
wheels_on                       float64
taxi_in                         float64
crs_arr_time                      int64
arr_time                        float64
crs_elapsed_time                  int64
actual_elapsed_time             float64
air_time                        float64
distance                          int64
dep_condition                    object
arr_condition                    object
month                             int64
average_seats                   float64
average_passengers              float64
avg_passenger_capacity_usage    float64
dtype: object

0         67.0
1          0.0
2          6.0
3         14.0
4          0.0
          ... 
119995    25.0
119996    15.0
119997     6.0
119998     0.0
119999    14.0
Name: arr_delay, Length: 117705, dtype: float64

In [24]:
flight_obj = flights.select_dtypes('object')

In [25]:
flight_dummies = pd.get_dummies(flight_obj)

In [26]:
flights_num = pd.merge(flights, flight_dummies, left_index=True, right_index=True)

In [27]:
flights_num = flights_num.select_dtypes(exclude=['object'])

In [28]:
scaler = StandardScaler()

In [29]:
scaler.fit(flights_num)

StandardScaler()

In [30]:
flights_scaled = scaler.transform(flights_num)

In [31]:
flights_scaled.shape

(110957, 795)

In [32]:

X_train, X_test, y_train, y_test= train_test_split(flights_scaled, y, test_size=0.2, random_state=7)

In [44]:
flights_num.shape

(11779, 749)

In [34]:
pca.fit(X_train)

PCA(n_components=15)

In [33]:
pca = PCA(n_components=15)

In [36]:
X_Train_PCA = pca.transform(X_train)

In [37]:
X_test_PCA = pca.transform(X_test)

In [33]:
regr = SVR(C= 10, epsilon= 1, kernel= 'rbf')


In [ ]:
regr.fit(X_train,y_train)

In [ ]:
y_pred = regr.predict()

In [32]:
parameters = {'kernel':('poly', 'rbf'), 'C':[0.001, 0.1,1, 10],'epsilon': [0.01, 0.05,0.1,0.2,0.3,1]}

In [33]:
reg_cv = GridSearchCV(regr,parameters)

In [34]:
reg_cv.fit(X_train,y_train)

GridSearchCV(estimator=SVR(),
             param_grid={'C': [0.001, 0.1, 1, 10],
                         'epsilon': [0.01, 0.05, 0.1, 0.2, 0.3, 1],
                         'kernel': ('poly', 'rbf')})

In [35]:
filename = 'reg_cv.sav'
pickle.dump(reg_cv, open(filename, 'wb'))

In [36]:
reg_cv.best_params_

{'C': 10, 'epsilon': 1, 'kernel': 'rbf'}

In [46]:
y_pred = regr.predict(X_test)

In [47]:
mean_squared_error(y_test, y_pred)

2866.614501275802

In [48]:
mean_absolute_error(y_test, y_pred)

14.493516770478461

In [49]:
r2_score(y_test, y_pred)

-0.057789413365094466

In [38]:
filename = 'svr_test.sav'
pickle.dump(regr, open(filename, 'wb'))

In [ ]:
svr = pickle.load(file_handle

In [47]:
from sklearn.linear_model import LinearRegression

In [52]:
lin_reg=LinearRegression()

In [53]:
lin_reg.fit(X_train,y_train)

LinearRegression()

In [55]:
y_pred = lin_reg.predict(X_test)

In [56]:
mean_absolute_error(y_test, y_pred)

672349965696.7395

In [57]:
mean_absolute_error(y_test, y_pred)

672349965696.7395

In [58]:
r2_score(y_test, y_pred)

-9.764910553223133e+23